In [1]:
import matplotlib.pyplot as plt
import numpy as np
import py21cmfast as p21c
import logging, os
from datetime import datetime
from numba import jit, njit

## Set logger to log caching activity

In [2]:
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

## Reset cache location 

In [3]:
p21c.config['direc'] = '/lustre/aoc/projects/hera/wchin/21cmFAST-cache'

## Colorbar function

In [4]:
def colorbar(mappable, plot_color='white'):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.ylabel('Neutral Fraction', color=plot_color)
    plt.tick_params(color=plot_color, labelcolor=plot_color)
    plt.sca(last_axes)
    
    ax.tick_params(color=plot_color, labelcolor=plot_color)
    
#     for spine in ax.spines.values():  # figure spine color
#         spine.set_edgecolor(plot_color)
    
    return cbar

## Cosmological Parameters (Default is used when no input is specified)

In [5]:
cosmo_params = p21c.CosmoParams()

## User Parameters, like box length, number of voxels (i.e. resolution) etc.

In [6]:
BOX_LEN=301  # 300, 301
HII_DIM=301  # 450, 301

user_params = p21c.UserParams(
    BOX_LEN=BOX_LEN,  # Box length in Mpc
    DIM=4*HII_DIM,      # Number of Voxels for hight resolution 
    HII_DIM=HII_DIM,  # Number of Voxels for low resolution 
    N_THREADS=os.cpu_count()
)

## Creating initial conditions box

In [ ]:
start_time = datetime.now()
print(f'Excution qued at {start_time}')

init_cond = p21c.initial_conditions(
    cosmo_params=cosmo_params,
    user_params=user_params,
    direc='/lustre/aoc/projects/hera/wchin/21cmFAST-cache'
)

end_time = datetime.now()
execution_time = end_time - start_time
print(f'Execution completed at {end_time}')
print(f'Execution time = {execution_time}')

Excution qued at 2020-12-26 15:06:53.427730


## CHAMP Bootcamp

In [ ]:
redshifts = np.array([8.5, 7.5, 6.5, 6.0])
R_BUBBLE_MAXES = np.array([15]*len(redshifts))
HII_EFF_FACTORS = np.array([30]*len(redshifts))

## Generate ionized boxes and total neutral fractions as a function of redshift

In [ ]:
progress_status = True

ionized_boxes = np.zeros((len(redshifts), HII_DIM, HII_DIM, HII_DIM))
total_neutral_fractions = np.zeros(len(redshifts))

# print progress and local time
if progress_status:
    start_time = datetime.now()
    current_time = start_time
    print(f'Progress = 0%, localtime = {start_time}')

for i, z in enumerate(redshifts):
    ionized_boxes[i] = p21c.ionize_box(
        redshift=z, 
        init_boxes=init_cond,
        astro_params={
            'HII_EFF_FACTOR': HII_EFF_FACTORS[i],
            'R_BUBBLE_MAX': R_BUBBLE_MAXES[i]
        }
    ).xH_box
    total_neutral_fractions[i] = np.mean(ionized_boxes[i])

    # print progress and local time
    if progress_status:
        previous_time = current_time
        current_time = datetime.now()
        loop_time = current_time - previous_time
        elapsed_time = current_time - start_time
        print(f'progress = {int(round((i+1)*100/len(redshifts)))}%, \
localtime = {current_time}, loopexecuted in {loop_time}, elapsedtime = {elapsed_time}')
        
total_neutral_fractions

## Make sure averaging region size is not larger than the box itself 

In [ ]:
def check_averaging_radius_limit(averaging_radius, box_length):
    # check to see if averaging region is larger than the box itself
    if averaging_radius > (box_length-1)/2:
        raise ValueError(f'Averaging_radius = {averaging_radius} > \
{(box_length-1)/2} = (Box Length-1)/2, averaging region is larger than the box itself.')

## Measure the distance of each voxel to the center

In [ ]:
@jit
def distance_from_coordinate(box_length):
    """
    Generate a cube of voxels.
    On each voxel, the distanace from the center is 
    calculated and the value is assigned to the voxel.
    jit by numba compiles what it can to machine code,
    the rest as python code.
    
    Parameters
    ----------
    box_length : int
        The length of each side of the cube.
        
    Returns
    -------
    distance : 3D-ndarray
        Cube of voxels with each voxel having its 
        distance from the center assigned to it.
    """
    # range of nummbers with 0 as the center    
    index = np.arange(-0.5*(box_length-1), 0.5*(box_length+1))
    # 3D mesh
    x_mesh, y_mesh, z_mesh = np.meshgrid(index, index, index, indexing='ij')
    # generating cube and computing distacne for each voxel
    distance = np.sqrt((x_mesh)**2 + (y_mesh)**2 + (z_mesh)**2)
    
    return distance

## Random Coordinate

In [ ]:
def random_cube_regions(box_length, number_of_coordinates, radius):
    """
    Selects a number cubical sub-regions
    within a larger cube at random positions.
    
    Parameters
    ----------
    box_length            : int
        Side length of the larger cube in
        arbitrary units of number of cells.
    number_of_coordinates : int
        Νumber of sub-regions to be selected.
    radius                : int
        Side of the smaller selected cubical 
        sub-region in arbitrary units of number of cells.
        Called radius because a sphere is
        often define within this smaller cube.
        
    Returns
    -------
    inds1 : ndarray dtype:int
        In 1D, the left bound that defines the cubical region.
    inds2 : ndarray dtype:int
        In 1D, the right bound that defines the cubical region.
    """
    
    np.random.seed()  # no entry: set seed to a randome number
#     np.random.seed(4)  # specifying seed for testing purposes

    # selecting 'number_of_coordiantes' random coordinates within larger cube
    coordinates = np.random.randint(0, box_length, size=(number_of_coordinates, 3))
    
    # cube indices 
    inds1 = (coordinates-radius).astype(int)
    inds2 = (coordinates+radius+1).astype(int)  # ending index is not inclusive
    

    return inds1, inds2

## Select a Smaller Cube with Sides 2R+1 Voxels, Centered about the Random Coordinate

In [ ]:
def slicing_the_cube(ind1, ind2, box):
    """
    Selects a smaller cubical sub-region within a larger cube called 'box'.
    Incoorporates periodic boundary conditions, i.e. Pac-Man effect.
    This function takes in a set of slicing indices of one particular randomly
    selected cubical sub-region and returns the selected smaller cube.
    
    Parameters
    ----------
    ind1 : 1D ndarray, dtype: int
        The left bounds of the selected region in 1D respectively.
    ind2 : 1D ndarray, dtype: int
        The right bounds of the selected region in 1D respectively.
    box  : 3D ndarray, dtype: float32
        Data cube, when the smaller cubical
        sub-regions are being selected from.
        
    Returns
    -------
    output_box : 3D ndarray, dtype: float32
        The selected cubical smaller sub-region
        within the larger data cube.
    """
        
    if ind1[0] < 0:  # periodic boundary conditions
        # region that went beyond the zeroth voxel face of the
        # cube is replaced by the region at the 'box_length'th
        # voxel face of the cube with the same size.
        x_inds = np.r_[(ind1[0]+len(box)):len(box), 0:ind2[0]]
    elif ind2[0] > len(box):
        # region that went beyond the 'box_length'th voxel face
        # of the cube is replaced by the region at the zeroth 
        # voxel face of the cube with the same size.
        x_inds = np.r_[ind1[0]:len(box), 0:(ind2[0]-len(box))]
    else:
        # selected voxel is perfectly in the larger data cube.
        x_inds = np.r_[ind1[0]:ind2[0]]

    if ind1[1] < 0:
        y_inds = np.r_[(ind1[1]+len(box)):len(box), 0:ind2[1]]
    elif ind2[1] > len(box):
        y_inds = np.r_[ind1[1]:len(box), 0:(ind2[1]-len(box))]
    else:
        y_inds = np.r_[ind1[1]:ind2[1]]

    if ind1[2] < 0:
        z_inds = np.r_[(ind1[2]+len(box)):len(box), 0:ind2[2]]
    elif ind2[2] > len(box):
        z_inds = np.r_[ind1[2]:len(box), 0:(ind2[2]-len(box))]
    else:
        z_inds = np.r_[ind1[2]:ind2[2]]
                    
    try:
        # box[indices]
        output_box = box[np.ix_(x_inds, y_inds, z_inds)]
        
    except IndexError:  # sample region larger than box.
        print(f'ind1 = {ind1}')  # print useful info
        print(f'ind2 = {ind2}')  # for debugging
        print(f'box length = {len(box)}')
        print(f'x_ind1 = {ind1[0]}')
        print(f'x_ind2 = {ind2[0]}')
        print(f'x_inds = {x_inds}')
        print(f'y_ind1 = {ind1[1]}')
        print(f'y_ind2 = {ind2[1]}')
        print(f'y_inds = {y_inds}')
        print(f'z_ind1 = {ind1[2]}')
        print(f'z_ind2 = {ind2[2]}')
        print(f'z_inds = {z_inds}')
        
    return output_box

## Gausssian Sphere Averaging

In [ ]:
def gaussian_sphere_average(
    distance_box, 
    radius, 
    input_box, 
    shell_num, 
    sigma_factor
):
    """
    Takes in a cube of voxels and defines 'shell_num'
    of concentric spherical shells and a core.
    A mean is taken over the voxels in each shell and the core.
    Then 'shell_num' equally spaced values are drawn as weights from 0 to 
    'sigma_factor' standard deviations of the Gaussian distribution.
    Each spherical shell and the core is assigned a weight, 
    The inner most core is weighted the most.
    The weights decrease moving outwards through the shells.
    And the outmost shell is weighted the least.
    The function then computes a weighted average over the shells and core.
    
    Parameters
    ----------
    distance_box : 3D ndarray, dtype: float32
        Cube of voxels with each voxel having its 
        distance from the center assigned to it.
        This function uses the distance box to
        set the condition required to slice
        the voxels in the spherical shells.
    radius : int
        Radius of the Guassian sphere, i.e. 
        the sphere complied by the center core 
        and the concentric spherical shells 
        in units of number of voxels.
    input_box : 
    
    
    
    """
    
    
    mean = np.zeros(shell_num)

    shell_radius_edges = np.linspace(0,1,shell_num+1)
    # sigma_factor number of sigmas the weighting goes out to, sigma = radius
    shell_center = 0.5*(shell_radius_edges[1:] + shell_radius_edges[:-1])*sigma_factor 
    weight = gaussian(x=shell_center)
    
    for ii in range(shell_num):
        condition = np.logical_and(
            distance_box <= shell_radius_edges[ii+1]*radius, 
            distance_box > shell_radius_edges[ii]*radius
        )
        mean[ii] = np.mean(input_box[condition])  # inside shell mean
        
    Gaussian_mean = np.average(mean, weights=weight)
    
    return Gaussian_mean

## Gaussian function

In [ ]:
@njit
def gaussian(x):  # μ=0, σ=1/sqrt(2), π=1
    """
    Gaussian distribution with amplitude=1,
    mu=0, standard deviation=1/sqrt(2), pi=1.
    @njit by numba compiles the code in machine code 
    instead of python, speeding up the computation.
    
    Parameters
    ----------
    x        : array_like
        Independent variable
        
    Returns
    -------
    gaussian : array_like
        Gaussian distribution array
    """
    gaussian = np.exp(-x**2)
    return gaussian

## Sphere Blurring Function

In [ ]:
@jit
def average_neutral_fraction_distribution(
    box, 
    radius, 
    iteration, 
    shell_num=6, 
    sigma_factor=1.4370397097748921*3, 
    blur_shape=None
):
    
    box = box.copy()  # make copy of input box to have a separate box
    
    mean_data = np.zeros(iteration)  # empty list for data collection
    
    
    if blur_shape == 'Gaussian_sphere':
        
        
# ====================================================================================================================
        # Radius Ratio 1
        radius = int(round(radius*1.4370397097748921*3))
        
        # Radius Ratio 2
#         radius = int(round(radius*((4/3/np.sqrt(np.pi))**(1/3))*13/4))  
            # 13/4 --> speculated correction factor
# ====================================================================================================================

        
        
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))
            
        # used as condition to define a sphere within a cube
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        
        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  

        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            # mean
            mean_data[i] = gaussian_sphere_average(
                dist_frm_coord_box, 
                radius, 
                cube_region_box, 
                shell_num, 
                sigma_factor
            )
        
    elif blur_shape == 'top_hat_sphere':
        
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))
            
        # used as condition to define a sphere within a cube
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        
        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  
        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            # mean
            mean_data[i] = top_hat_sphere_average(
                dist_frm_coord_box, 
                radius, 
                cube_region_box
            )
            
    elif blur_shape == 'top_hat_cube':
                
        # ratio determiend by equating the volumes of cube & sphere
        radius = int(round((radius*((4*np.pi/3)**(1/3))-1)/2))  

        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))

        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  
        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            # mean
            mean_data[i] = top_hat_cube_average(cube_region_box)
    else:
        
        print('Blurring shape assumed to be a Gaussian sphere with 4 shells \
              weighted by equally spaced values from 0 sigma to 4 sigma.')
                
        
# ====================================================================================================================
        # Radius Ratio 1
        radius = int(round(radius*1.4370397097748921*3))
        
        # Radius Ratio 2
#         radius = int(round(radius*((4/3/np.sqrt(np.pi))**(1/3))*13/4))  
            # 13/4 --> speculated correction factor
# ====================================================================================================================
        
        
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))

        # used as condition to define a sphere within a cube
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)

        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  
        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            mean_data[i] = gaussian_sphere_average(
                dist_frm_coord_box, 
                radius, 
                cube_region_box, 
                shell_num, 
                sigma_factor
            )
            # mean
            
    return mean_data

## Generate Average Neutral Fraction Distributions as a function of redshift

In [ ]:
def generate_distributions(
    boxes,
    radii=np.arange(8, 17, 1),
    iterations=int(10**3),
    sigma_factor=1.4370397097748921*3,
    shell_number=6,
    progress_status=True
):

    gaussians = np.zeros((len(boxes), len(radii), iterations))
    
    # print progress and local time
    if progress_status:
        start_time = datetime.now()
        current_time = start_time
        print(f'Progress = 0%, localtime = {start_time}')

    for i, box in enumerate(boxes):
               
        for ii, radius in enumerate(radii):
                        
            gaussians[i, ii, :] = average_neutral_fraction_distribution(
                box=box,
                radius=radius,
                iteration=iterations,
                sigma_factor=sigma_factor,
                shell_num=shell_number,
                blur_shape='Gaussian_sphere'
            )
        # print progress and local time
        if progress_status:
            previous_time = current_time
            current_time = datetime.now()
            loop_time = current_time - previous_time
            elapsed_time = current_time - start_time
            print(f'progress = {int(round((i+1)*100/len(boxes)))}%, \
localtime = {current_time}, loopexecuted in {loop_time}, elapsedtime = {elapsed_time}')
    
    return gaussians

In [ ]:
radii = np.array([16, 15])  # array([34, 32, 30,... 6, 4]) units: voxels (51, 3, -3)
iterations = int(1e5)

gaussians = generate_distributions(
    boxes=np.array([ionized_boxes[1]]),
    radii=radii,
    iterations=iterations,
    sigma_factor=1.4370397097748921*3
)

In [19]:
np.save('/lustre/aoc/projects/hera/wchin/gaussians_z7.5_r16,15', gaussians)

In [20]:
radii = np.array([10])  # array([34, 32, 30,... 6, 4]) units: voxels (51, 3, -3)
iterations = int(1e5)

gaussians = generate_distributions(
    boxes=np.array([ionized_boxes[-1]]),
    radii=radii,
    iterations=iterations,
    sigma_factor=1.4370397097748921*3
)

Progress = 0%, localtime = 2020-12-23 13:37:22.636248
progress = 100%, localtime = 2020-12-23 13:53:20.062243, loopexecuted in 0:15:57.425995, elapsedtime = 0:15:57.425995


In [21]:
np.save('/lustre/aoc/projects/hera/wchin/gaussians_z6.0_r10', gaussians)